# Проект - выбор региона бурения

Для добывающей компании необходимо решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Нам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Нужно построить модель для определения региона, где добыча принесёт наибольшую прибыль. Будем анализировать возможную прибыль и риски техникой Bootstrap.

## Последовательность выполнения проекта

- Загрузим и подготовим данные.

- Обучим и проверим модель для каждого региона:

    - Разделим данные на обучающую и валидационную выборки в соотношении 75:25.
    - Обучим модель и сделаем предсказания на валидационной выборке.
    - Сохраним предсказания и правильные ответы на валидационной выборке.
    - Выведем на экран средний запас предсказанного сырья и RMSE модели.
    - Проанализируем результаты.
    
- Подготовимся к расчёту прибыли:

    - Все ключевые значения для расчётов сохраним в отдельных переменных.
    - Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.
    - Напишем выводы по этапу подготовки расчёта прибыли.
    
- Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

    - Выберем скважины с максимальными значениями предсказаний.
    - Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
    - Рассчитаем прибыль для полученного объёма сырья.
    
- Посчитаем риски и прибыль для каждого региона:

    - Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    - Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
    - Сделаем выводы: предложим регион для разработки скважин и обоснуем выбор.

## Описание данных

- Данные геологоразведки трёх регионов находятся в соотвествующих файлах
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# Выполнение проекта

## Загрузка и общий анализ данных

### Загрузка данных

In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from math import sqrt
import numpy as np
from scipy import stats as st


try:
    df_0=pd.read_csv('C:/Users/kaz-106/YandexDisk/Py-projects/02_Practicum projects/07_Machine_learning_in_business/geo_data_0.csv')
except:
    df_0=pd.read_csv('/datasets/geo_data_0.csv')
    
try:
    df_1=pd.read_csv('C:/Users/kaz-106/YandexDisk/Py-projects/02_Practicum projects/07_Machine_learning_in_business/geo_data_1.csv')
except:
    df_1=pd.read_csv('/datasets/geo_data_1.csv')    
    
try:
    df_2=pd.read_csv('C:/Users/kaz-106/YandexDisk/Py-projects/02_Practicum projects/07_Machine_learning_in_business/geo_data_2.csv')
except:
    df_2=pd.read_csv('/geo_data_2.csv')    
    
df_all=[df_0,df_1,df_2]

for data in df_all:
    print(data.info())
    print(data.dtypes)
    display(data.head())

    
   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Проверка пропуски

- Из предыдущего шага видно, что в данных нет пропусков

### Проверка на дубликаты.

- Есть несколько повторяющихся id - удалим их

In [2]:
for data in df_all:
    print(data['id'].duplicated().sum())
   

10
4
4


In [3]:
df_0=df_0.drop_duplicates(subset=['id'])
df_1=df_1.drop_duplicates(subset=['id'])
df_2=df_2.drop_duplicates(subset=['id'])
df_all=[df_0,df_1,df_2]

In [4]:
for data in df_all:
    print(data['id'].duplicated().sum())

0
0
0


### Удаление лишних столбцов

- Столбец Id не нужен для обучения модели - удалим его

In [5]:
df_0=df_0.drop('id',axis=1)

df_1=df_1.drop('id',axis=1)

df_2=df_2.drop('id',axis=1)


## Обучение и проверка модели по регионам

### Разделение выборок

- Разделим выборки на обучающие и валидационные

In [6]:
train_0, valid_0 = train_test_split(df_0, test_size=0.25,train_size=0.75,random_state=12345)
train_1, valid_1 = train_test_split(df_1, test_size=0.25,train_size=0.75,random_state=12345)
train_2, valid_2 = train_test_split(df_2, test_size=0.25,train_size=0.75,random_state=12345)

df_all_splitted=[train_0, valid_0, train_1, valid_1, train_2, valid_2]

for data in df_all_splitted:
    print(data.shape)
    print()


(74992, 4)

(24998, 4)

(74997, 4)

(24999, 4)

(74997, 4)

(24999, 4)



### Выделение признаков

- Выделим признаки из обучающих и валидационных наборов по всем регионам

In [7]:
features_train_0 = train_0.drop('product', axis=1)
target_train_0 = train_0['product']

features_valid_0 = valid_0.drop('product', axis=1)
target_valid_0 = valid_0['product'] 



features_train_1 = train_1.drop('product', axis=1)
target_train_1 = train_1['product']

features_valid_1 = valid_1.drop('product', axis=1)
target_valid_1 = valid_1['product'] 



features_train_2 = train_2.drop('product', axis=1)
target_train_2 = train_2['product']

features_valid_2 = valid_2.drop('product', axis=1)
target_valid_2 = valid_2['product'] 

### Обучение моделей

#### Регион 1

In [8]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0)
mse_0 = mean_squared_error(target_valid_0, predicted_valid_0)
rms_0 = sqrt(mse_0)

print("Регион 1")
print("Модель линейной регрессии")
print("MSE =", mse_0)
print("RMSE =", rms_0)
print("Среднее =", predicted_valid_0.mean())




Регион 1
Модель линейной регрессии
MSE = 1432.8895312377324
RMSE = 37.853527328872964
Среднее = 92.78915638280621


#### Регион 2

In [9]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)
mse_1 = mean_squared_error(target_valid_1, predicted_valid_1)
rms_1 = sqrt(mse_1)

print("Регион 2")
print("Модель линейной регрессии")
print("MSE =", mse_1)
print("RMSE =", rms_1)
print("Среднее =", predicted_valid_1.mean())

Регион 2
Модель линейной регрессии
MSE = 0.7957697318650299
RMSE = 0.8920592647717022
Среднее = 69.17831957030432


#### Регион 3

In [10]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

predicted_valid_2 = model_2.predict(features_valid_2)
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
rms_2 = sqrt(mse_2)


print("Регион 3")
print("Модель линейной регрессии")
print("MSE =", mse_2)
print("RMSE =", rms_2)
print("Среднее =", predicted_valid_2.mean())

Регион 3
Модель линейной регрессии
MSE = 1606.0738119304278
RMSE = 40.07585073246016
Среднее = 94.86572480562035


### Выводы

- Наибольшими запасами обладает регион 3, но в тоже время у него значительно большая ошибка
- Чуть меньшие запасы в регионе 1, при меньшей ошибке
- Регион 2 обладает самыми маленькими запасами

## Подготовка к расчету прибыли

### Ввод общих переменных

In [11]:
HOLES_TOTAL=500 # общее количество исследуемых точек в регионе

HOLES_SELECTION=200 # количество точек выбираемых для разработки

TOTAL_BUDGET=10000000000 # общий бюджет на разработку скважин в регионе

BARREL_REVENUE = 450 # доход с одного барреля

PRODUCT_REVENUE = 450000 # доход с единицы продукта

RISK_LEVEL = 0.025 # допустимый уровень риска возникновения убытков

### Рассчет достаточного объёма сырья для безубыточной разработки новой скважины

In [12]:
one_hole_budget = TOTAL_BUDGET/HOLES_SELECTION

one_hole_product_need = one_hole_budget/PRODUCT_REVENUE
print('Минимальный необходимый уровень запасов в скважине для разработки:',round(one_hole_product_need,1))

mean_oil_reserves=[predicted_valid_0.mean(),predicted_valid_1.mean(),predicted_valid_2.mean()]
print()
print('Оценка рентабельности регионов по средним запасам')
print()
def reserves_check(mean_reserves):
    for i in range(len(mean_oil_reserves)):
        if mean_reserves[i] < one_hole_product_need:
            print(f'Запас сырья в регионе {i+1} меньше необходимого на {round(one_hole_product_need - mean_oil_reserves[i], 1)}')
        else:
            print(f'Запас сырья в регионе {i+1} большин необходимого на {round(mean_oil_reserves[i] - one_hole_product_need, 1)}')

reserves_check(mean_oil_reserves)




Минимальный необходимый уровень запасов в скважине для разработки: 111.1

Оценка рентабельности регионов по средним запасам

Запас сырья в регионе 1 меньше необходимого на 18.3
Запас сырья в регионе 2 меньше необходимого на 41.9
Запас сырья в регионе 3 меньше необходимого на 16.2


- Среднее значение уровня запасов во всех трех регионах ниже минимально необходимого уровня,

## Функция для расчета прибыли и рисков

- Напишем соотвествущую функцию
- Будем подавать на вход функции предсказания модели и целевые выборки
- Выберем 200 самых богатых скважин

In [13]:
def profit(target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    predictions_selection = predictions.sort_values(ascending=False)[:HOLES_SELECTION].index
    selection = target[predictions_selection]
    profit = (selection.sum() * PRODUCT_REVENUE) - TOTAL_BUDGET
    return round(profit, 1)

In [14]:
print('Прибыль в регионе 1:',profit(target_valid_0, predicted_valid_0))

Прибыль в регионе 1: 3365187237.7


In [15]:
print('Прибыль в регионе 2:',profit(target_valid_1, predicted_valid_1))

Прибыль в регионе 2: 2415086696.7


In [16]:
print('Прибыль в регионе 3:',profit(target_valid_2, predicted_valid_2))

Прибыль в регионе 3: 2501283853.3


## Bootstrap для выбора региона

### Общая функция

In [17]:
def profit_b(target, probabilities, count, sample_size):
    state = np.random.RandomState(12345)
    values = []
    for i in range(count):
        target_subsample = target.sample(n=sample_size, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        
        values.append(profit(target_subsample, probs_subsample))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean()
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    quantile_interval = (lower, upper)
    return round(mean,1), round(risk*100, 1), quantile_interval

### Регион 1

In [18]:
mean_0, risk_0, interval_0,  = profit_b(target_valid_0.reset_index(drop=True), predicted_valid_0, 1000, 500)
print('Регион 1')
print()
print('Средняя выручка = ', mean_0)
print('95 %ый доверительный интервал по квантилям = ', interval_0)
print(f'Риск убытков: {risk_0}%')

Регион 1

Средняя выручка =  380613470.0
95 %ый доверительный интервал по квантилям =  (-142942739.71, 890976833.81)
Риск убытков: 7.7%


### Регион 2

In [19]:
mean_1, risk_1, interval_1,  = profit_b(target_valid_1.reset_index(drop=True), predicted_valid_1, 1000, 500)
print('Регион 2')
print()
print('Средняя выручка = ', mean_1)
print('95 %ый доверительный интервал по квантилям = ', interval_1)
print(f'Риск убытков: {risk_1}%')

Регион 2

Средняя выручка =  478484939.6
95 %ый доверительный интервал по квантилям =  (89525124.9925, 866604745.5099999)
Риск убытков: 1.2%


### Регион 3

In [20]:
mean_2, risk_2, interval_2,  = profit_b(target_valid_2.reset_index(drop=True), predicted_valid_2, 1000, 500)
print('Регион 3')
print()
print('Средняя выручка = ', mean_2)
print('95 %ый доверительный интервал по квантилям = ', interval_2)
print(f'Риск убытков: {risk_2}%')

Регион 3

Средняя выручка =  330546258.0
95 %ый доверительный интервал по квантилям =  (-190365733.89499998, 840467403.6174998)
Риск убытков: 11.3%


## Выводы

- По результатам применения техники Bootstrap следует разрабатывать скважины в регионе 2. Там наибольшая средняя выручка, лучшие значения 95% доверительного интервала и низкий риск убытков